# Voltage Conversion from Full Cell to Half-Cell Potentials

This notebook demonstrates how to convert full cell voltage measurements to half-cell potentials (vs SHE and vs RHE) using calibration data from electrochemical measurements in a three-electrode configuration.

## Methodology

The conversion accounts for:
- Membrane overpotential and ionic resistance
- Nernstian pH gradient effects
- Reference electrode potential corrections
- Current density-dependent ohmic losses

**Reference:** Arabyarmohammadi, F. et al. Voltage distribution within carbon dioxide reduction electrolysers. *Nature Sustainability* (2025) - https://www.nature.com/articles/s41893-025-01643-4


## Import Required Libraries


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import linregress
import pandas as pd


## Calibration Data and Experimental Conditions

**Experiment conditions:** Neutral CO₂RR in 4cm² cell, Sputtered Copper Catalyst, 0.1M Bicarbonate - ref electrode (3M KCl) 230mV vs SHE


In [2]:
# Experiment conditions
ref_pot = 0.23  # V Ag/AgCl electrode
cathode_pH = 12.5
anode_pH = 3
Nern_pH_loss = (cathode_pH - anode_pH) * 0.059 
geo_area = 4  # cm²
membrane_loss = 0.1  # V
cathode_thermo = +0.08
anode_thermo = +1.23
thermo_pot = anode_thermo - cathode_thermo

print(f"Reference electrode potential: {ref_pot} V vs SHE")
print(f"Cathode pH: {cathode_pH}, Anode pH: {anode_pH}")
print(f"Nernstian pH loss: {Nern_pH_loss:.3f} V")
print(f"Geometric area: {geo_area} cm²")
print(f"Membrane loss: {membrane_loss} V")
print(f"Thermodynamic potentials - Cathode: {cathode_thermo} V, Anode: {anode_thermo} V")


Reference electrode potential: 0.23 V vs SHE
Cathode pH: 12.5, Anode pH: 3
Nernstian pH loss: 0.560 V
Geometric area: 4 cm²
Membrane loss: 0.1 V
Thermodynamic potentials - Cathode: 0.08 V, Anode: 1.23 V


In [3]:
# Measured values from calibration work
j = np.array([50, 100, 200])  # mA/cm²
cathode_pot = np.array([-1.62, -2.0, -2.3])  # V
cathode_R = np.array([0.48, 0.34, 0.3])  # Ω
anode_pot = np.array([1.3, 1.35, 1.4])  # V
anode_R = np.array([0, 0, 0])  # almost negligible
fullcell_pot = np.array([3, 3.4, 3.7])  # V
fullcell_R = np.array([0.47, 0.35, 0.3])  # Ω

print("Calibration measurements:")
print(f"Current density (mA/cm²): {j}")
print(f"Cathode potential (V): {cathode_pot}")
print(f"Cathode resistance (Ω): {cathode_R}")
print(f"Anode potential (V): {anode_pot}")
print(f"Full cell potential (V): {fullcell_pot}")
print(f"Full cell resistance (Ω): {fullcell_R}")


Calibration measurements:
Current density (mA/cm²): [ 50 100 200]
Cathode potential (V): [-1.62 -2.   -2.3 ]
Cathode resistance (Ω): [0.48 0.34 0.3 ]
Anode potential (V): [1.3  1.35 1.4 ]
Full cell potential (V): [3.  3.4 3.7]
Full cell resistance (Ω): [0.47 0.35 0.3 ]


## Core Conversion Functions


In [4]:
def she2rhe(ushe, pH, ref_pot):
    """Convert SHE potential to RHE potential."""
    ushe = ushe + ref_pot + (0.059 * pH)
    return ushe

def rhe2she(urhe, pH, ref_pot):
    """Convert RHE potential to SHE potential."""
    urhe = urhe - ref_pot - (0.059 * pH)
    return urhe

def correct_potential(pot, R, pH, j, area, ref_pot):
    """Correct potential for ohmic losses."""
    if pot < 0:
        corrected_pot = she2rhe(pot + j/1000 * area * R, pH, ref_pot)
    else:
        corrected_pot = she2rhe(pot - j/1000 * area * R, pH, ref_pot)
    return corrected_pot

def get_overpotential(pot, pot_theory):
    """Calculate overpotential."""
    overpot = abs(pot - pot_theory)
    return overpot


## Data Processing and Correction


In [5]:
# Process calibration data
n = len(cathode_pot)
cathode_pot_corr = np.zeros(n)
anode_pot_corr = np.zeros(n)
cathode_overpot = np.zeros(n)
anode_overpot = np.zeros(n)
fullcell_pot_corr = np.zeros(n)

for i in range(n):
    cathode_pot_corr[i] = correct_potential(cathode_pot[i], cathode_R[i], cathode_pH, j[i], geo_area, ref_pot)
    anode_pot_corr[i] = correct_potential(anode_pot[i], anode_R[i], anode_pH, j[i], geo_area, ref_pot)
    cathode_overpot[i] = get_overpotential(cathode_pot_corr[i], 0.08)
    anode_overpot[i] = get_overpotential(anode_pot_corr[i], 1.23)
    fullcell_pot_corr[i] = fullcell_pot[i] - fullcell_R[i] * j[i] / 1000 * geo_area

print('Corrected potentials:')
print(f'Cathode corrected potential (V vs RHE): {cathode_pot_corr}')
print(f'Anode corrected potential (V vs RHE): {anode_pot_corr}')
print(f'Cathode overpotential (V): {cathode_overpot}')
print(f'Anode overpotential (V): {anode_overpot}')


Corrected potentials:
Cathode corrected potential (V vs RHE): [-0.5565 -0.8965 -1.0925]
Anode corrected potential (V vs RHE): [1.707 1.757 1.807]
Cathode overpotential (V): [0.6365 0.9765 1.1725]
Anode overpotential (V): [0.477 0.527 0.577]


## Linear Regression Functions


In [6]:
def lin_fxn(x, a, b):
    """Linear function for curve fitting."""
    return a * x + b

def fit_lin(X, Y):
    """Fit linear function to data."""
    params, covariance = curve_fit(lin_fxn, X, Y)
    a_fit, b_fit = params
    return (a_fit, b_fit)

def est_x(x, X, Y):
    """Estimate x value using linear fit."""
    fit = fit_lin(X, Y)
    x = fit[0] * x + fit[1]
    return x


## Main Conversion Functions


In [7]:
def cell2rhe(vcell, X, Y):
    """Convert full cell voltage to RHE potential."""
    vrhe = est_x(vcell, X, Y)
    return vrhe

def fullcell2halfcell(vcell):
    """
    Main function to convert a voltage value from full cell to half cell vs SHE or RHE.
    
    Parameters:
    -----------
    vcell : float
        Full cell voltage (V)
        
    Returns:
    --------
    ushe : float
        Half-cell potential vs SHE (V)
    urhe : float
        Half-cell potential vs RHE (V)
    """
    cali_dict = load_calibration_data_for_voltage_conversion()
    urhe = cell2rhe(vcell, cali_dict['measurements']['fullcell pot'], cali_dict['extracted params']['cathode pot corr'])
    ushe = rhe2she(urhe, cali_dict['conditions']['cathode pH'], cali_dict['conditions']['ref pot'])
    return ushe, urhe

def load_calibration_data_for_voltage_conversion():
    """Load calibration data for voltage conversion."""
    conditions_dict = {
        'ref pot': ref_pot,
        'cathode pH': cathode_pH,
        'anode pH': anode_pH,
        'Nern pH loss': Nern_pH_loss,
        'geo area': geo_area,
        'membrane loss': membrane_loss,
        'cathod thermo': cathode_thermo,
        'anode thermo': anode_thermo,
        'thermo pot': thermo_pot,
    }
    measurements_dict = {
        'j': j, 'cathode pot': cathode_pot, 'cathode R': cathode_R, 
        'anode pot': anode_pot, 'anode R': anode_R, 
        'fullcell pot': fullcell_pot, 'fullcell R': fullcell_R,
    }
    data_dict = {
        'cathode pot corr': cathode_pot_corr, 'anode pot corr': anode_pot_corr,
        'cathode overpot': cathode_overpot, 'anode overpot': anode_overpot, 
        'fullcell pot corr': fullcell_pot_corr    
    }
    return {'measurements': measurements_dict, 'conditions': conditions_dict, 'extracted params': data_dict}


## Working Example: Convert Sample Voltages


In [8]:
# Example: Convert some sample full cell voltages
sample_voltages = [2.8, 3.2, 3.5, 3.8, 4.0]

print("Voltage Conversion Example:")
print("=" * 60)
print(f"{'Full Cell (V)':<12} {'vs SHE (V)':<12} {'vs RHE (V)':<12}")
print("-" * 60)

for vcell in sample_voltages:
    ushe, urhe = fullcell2halfcell(vcell)
    print(f"{vcell:<12.1f} {ushe:<12.3f} {urhe:<12.3f}")


Voltage Conversion Example:
Full Cell (V) vs SHE (V)   vs RHE (V)  
------------------------------------------------------------
2.8          -1.380       -0.412      
3.2          -1.688       -0.720      
3.5          -1.919       -0.951      
3.8          -2.150       -1.182      
4.0          -2.304       -1.336      


## Validation: Compare Estimated vs True Values


In [9]:
print("Validation of Conversion Accuracy:")
print("=" * 80)

for i in range(len(fullcell_pot)):
    val = fullcell_pot[i]
    est_val = cell2rhe(fullcell_pot[i], fullcell_pot, cathode_pot_corr)
    true_val = cathode_pot_corr[i]
    error = (true_val - est_val) / true_val * 100
    print(f'Vcell: {val:.2f} V \t Estimated VRHE: {est_val:.3f} V \t True VRHE: {true_val:.3f} V \t Error: {error:.2f}%')
    print('-' * 80)


Validation of Conversion Accuracy:
Vcell: 3.00 V 	 Estimated VRHE: -0.566 V 	 True VRHE: -0.557 V 	 Error: -1.72%
--------------------------------------------------------------------------------
Vcell: 3.40 V 	 Estimated VRHE: -0.874 V 	 True VRHE: -0.896 V 	 Error: 2.49%
--------------------------------------------------------------------------------
Vcell: 3.70 V 	 Estimated VRHE: -1.105 V 	 True VRHE: -1.092 V 	 Error: -1.17%
--------------------------------------------------------------------------------


## Usage Instructions

To use these functions in your own analysis:

1. **Import the functions** from this notebook
2. **Call `fullcell2halfcell(vcell)`** with your full cell voltage
3. **The function returns** `(ushe, urhe)` - half-cell potentials vs SHE and RHE

### Example:
```python
# Convert a full cell voltage of 3.5 V
ushe, urhe = fullcell2halfcell(3.5)
print(f"Half-cell potential vs SHE: {ushe:.3f} V")
print(f"Half-cell potential vs RHE: {urhe:.3f} V")
```

### For batch conversion:
```python
# Convert an array of voltages
voltages = [2.8, 3.2, 3.5, 3.8, 4.0]
she_values = []
rhe_values = []

for v in voltages:
    ushe, urhe = fullcell2halfcell(v)
    she_values.append(ushe)
    rhe_values.append(urhe)

print(f"SHE values: {she_values}")
print(f"RHE values: {rhe_values}")
```


## References

1. Arabyarmohammadi, F. et al. Voltage distribution within carbon dioxide reduction electrolysers. *Nature Sustainability* (2025) - https://www.nature.com/articles/s41893-025-01643-4

2. This methodology follows established protocols for accurate half-cell potential determination in CO₂ reduction electrolyzers.
